In [12]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#from scipy.stats import norm
#from scipy.stats import linregress
#import statsmodels.formula.api as smf
import csv
import scipy.optimize as opt
import pandas_datareader as pdr
from datetime import datetime


In [2]:
# function to read in data from a csv
def readPricesCSV(file):
    df = pd.read_csv(file)
    dfPrices = df["Close"]
    return dfPrices

In [17]:
# function to read in stock prices from yahoo finance in real time
def readPricesYF(start_date, end_date, tickers):
    stockPrices = pdr.get_data_yahoo(tickers, start=start_date, end=end_date)   # read in the data
    stockPrices = stockPrices.filter(like='Adj Close')   # tickers are columns and data values are adjusted closing prices with the date as the index
    stockPrices.columns = tickers   # change column names to be their tickers
    stockPrices = stockPrices.stack().swaplevel().sort_index().reset_index()    # make the data tall and clean it up so it is easier to perform analysis on
    stockPrices.columns = ['Firm','Date','Adj Close']     # rename the columns
#    stockPrices['Return'] = stockPrices.groupby('Firm')['Adj Close'].pct_change()    # get the daily returns for each ticker
    return stockPrices


In [16]:
# the objective function
def obj(weights, means, risk_level, covMatr):
    
    #get the return measure given by the weights and mean
    for weight,mean in weights,means:
        sum_return += weight*mean
    num_stocks = len(means)
    
    #get the risk measure given by the weights
    risk_measure = np.matmul(np.matmul(np.transpose(np.array(weights)), covMatr), np.array(weights))

    #return the objective function value
    return -(sum_return - (risk_level * risk_measure))


In [206]:
#choose which stocks to include in the optimization of the portfolio (list the tickers)
tickers = ['MSFT', 'AAPL', 'AMZN', 'NFLX']
tickers = sorted(tickers)

#set the start and end dates- choose last six months in this case
start_date = datetime(2019, 10, 30) #year, month, day
end_date = datetime(2020,10,29) #year, month, day

stockPrices = readPricesYF(start_date, end_date, tickers)    # use the function defined above to get data from Yahoo Finance


In [207]:
stockPrices

,Firm,Date,Adj Close
0,AAPL,2019-10-30,60.024124
1,AAPL,2019-10-31,61.381241
2,AAPL,2019-11-01,63.123295
3,AAPL,2019-11-04,63.537827
4,AAPL,2019-11-05,63.446533
...,...,...,...
1007,NFLX,2020-10-23,488.279999
1008,NFLX,2020-10-26,488.239990
1009,NFLX,2020-10-27,488.929993
1010,NFLX,2020-10-28,486.239990


In [210]:
# get the optimization ready

num_stocks = len(tickers)
stock_prices_format = stockPrices.set_index(['Date', 'Firm']).unstack()  #.T
stock_prices_format.reset_index()
#stock_prices_format.iloc[:,[0,1,2,3]]
#stock_prices_format.iloc[:,[0]]
#prices = stock_prices_format.iloc[:,[0,1,2,3]]
#prices = stockPrices.reset_index().drop(['Date','index'], axis=1).set_index('Firm').unstack()   #.drop('index', axis=1)
#prices

prices_per_stock = stock_prices_format.shape

#np.array(stock_prices_format.iloc[:,[0,1,2,3]].reset_index().drop('Date', axis=1)[0:])#.reshape(prices_per_stock, num_stocks)

#prices = stock_prices_format.drop('Date', axis=1)

np.array(stock_prices_format.iloc[:,[0,1,2,3]]) #.flatten()#.reshape([prices_per_stock, num_stocks])

#cov_matr = np.cov(np.array(stock_prices_format.iloc[:,[0,1,2,3]]))
#cov_matr

stockPrices.drop('Date', axis=1).set_index('Firm').stack()

np.array(stock_prices_format.iloc[:,[0,1,2,3]].T)

array([[  60.02412415,   61.38124084,   63.12329483, ...,  116.22550964,
         110.84284973,  114.94961548],
       [1779.98999023, 1776.66003418, 1791.43994141, ..., 3286.33007812,
        3162.7800293 , 3211.01000977],
       [ 142.28117371,  141.06111145,  141.40550232, ...,  212.204422  ,
         201.68623352,  203.7162323 ],
       [ 291.45001221,  287.41000366,  286.80999756, ...,  488.92999268,
         486.23999023,  504.20999146]])